In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv 
import edgar
import os
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

Get CIK list of all investment companies

In [2]:
df_MF = pd.read_csv(r'https://goz39a.s3.eu-central-1.amazonaws.com/13F/CIK_list.csv')
df_MF['Entity Name'] = df_MF['Entity Name']
#[i.find(isin(t)) for i in df_MF.loc[df_MF['Entity Name']]]
df_MF

,Unnamed: 0,CIK Number,Entity Name,Address_1,City,State,Zip Code
0,0,2110,COLUMBIA ACORN TRUST,227 W MONROE STE 3000,CHICAGO,IL,60606
1,1,2646,Voya INTERMEDIATE BOND PORTFOLIO,ING FUNDS SERVICES,SCOTTSDALE,AZ,85258
2,2,2663,VOYA GOVERNMENT MONEY MARKET PORTFOLIO,"7337 E. DOUBLETREE RANCH ROAD, STE 100",SCOTTSDALE,AZ,85258
3,3,2664,Voya VARIABLE FUNDS,"7337 E. DOUBLETREE RANCH ROAD, STE 100",SCOTTSDALE,AZ,85258
4,4,2691,LORD ABBETT AFFILIATED FUND INC,90 HUDSON STREET,JERSEY CITY,NJ,07302
...,...,...,...,...,...,...,...
2146,2146,1810747,Simplify Exchange Traded Funds,154 W 14TH ST,NEW YORK,NY,10011
2147,2147,1811168,2nd Vote Funds,PO BOX 2916,HENDERSONVILLE,TN,37077
2148,2148,1815558,Waycross Independent Trust,"C/O ULTIMUS FUND SOLUTIONS, LLC",CINCINNATI,OH,45246
2149,2149,1815608,Modern Capital Funds Trust,1050 JOHNNIE DODDS BLVD,MT. PLEASANT,SC,29465


download all filings from 2019-2020-2021

In [3]:
path= r'c:/Users/ASUS/Desktop/filing_history_bis2'
edgar.download_index(path, 2019, skip_all_present_except_last=False)

select the URL to the edgar archive for the 13F-HR filings of 2019 until q1 of 2021

In [4]:
def get_urls(path_download_index):
    with open(path_download_index,'r') as file:
        tsvreader = csv.reader(file, delimiter="|")
        urls= []
        file = []
        for line in tsvreader:
            if line[1:][1] == '13F-HR':
                adress=line[-1]
                url =  'https://www.sec.gov/Archives/' + adress
                holding = line[1]
                urls.append((holding.lower(), url))
    return urls

In [5]:
path= r'c:/Users/ASUS/Desktop/filing_history_bis2'

urls_2019_1 = get_urls(path + os.sep + '2019-QTR1.tsv')
urls_2019_2 = get_urls(path + os.sep + '2019-QTR2.tsv')
urls_2019_3 = get_urls(path + os.sep + '2019-QTR3.tsv')
urls_2019_4 = get_urls(path + os.sep + '2019-QTR4.tsv')
urls_2020_1 = get_urls(path + os.sep + '2020-QTR1.tsv')
urls_2020_2 = get_urls(path + os.sep + '2020-QTR2.tsv')
urls_2020_3 = get_urls(path + os.sep + '2020-QTR3.tsv')
urls_2020_4 = get_urls(path + os.sep + '2020-QTR4.tsv')
urls_2021 = get_urls(path + os.sep + '2021-QTR1.tsv')

Select only investment companies with 13F-HR filings in all quarters

In [6]:
urls_2019_1names, _ = zip(*urls_2019_1)
urls_2019_2names, _ = zip(*urls_2019_2)
urls_2019_3names, _ = zip(*urls_2019_3)
urls_2019_4names, _ = zip(*urls_2019_4)
urls_2020_1names, _ = zip(*urls_2020_1)
urls_2020_2names, _ = zip(*urls_2020_2)
urls_2020_3names, _ = zip(*urls_2020_3)
urls_2020_4names, _ = zip(*urls_2020_4)
urls_2021_names, _ =zip(*urls_2021)

In [7]:
companiesBoth = [i for i in (urls_2019_1names and urls_2019_2names and urls_2019_3names and urls_2019_4names and urls_2020_1names and urls_2020_2names and urls_2020_3names and urls_2020_4names and urls_2021_names  )]

In [8]:
len(companiesBoth)

5970

List of biggest investment companies from https://mutualfunddirectory.org/latest-directory-ranking-here/

In [9]:
URL = 'https://mutualfunddirectory.org/latest-directory-ranking-here/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_="tablepress-id-109")
for company in table.find_all('tbody'):
    rows= company.find_all('tr')
    companies =[]
    for row in rows:
        companies.append(row.find('td', class_="column-2").text)

companies = companies[:-1]
companies = [i.lower() for i in companies]

From companies with filings in each quarter, only select biggest companies

In [10]:
urls_list=[]
period = [urls_2019_1, urls_2019_2, urls_2019_3, urls_2019_4, urls_2020_1, urls_2020_2, urls_2020_3, urls_2020_4, urls_2021]
for quarter in period:
    urls = []
    companies_quarter = [i for i in quarter if i[0] in companiesBoth]
    for company_quarter in companies_quarter:
        if len(company_quarter[0].split())>1:
            if company_quarter[0].split()[0] in companies or company_quarter[0].split()[1] in companies:
                urls.append(company_quarter[1])
        else:
             if company_quarter[0].split()[0] in companies:
                urls.append(company_quarter[1])
    urls_list.append(urls)

In [11]:
unique_companies=[urls_list[1][j].split('data/')[1].split('/')[0] for j in range(len(urls_list[1]))]
unique_companies

['1027796',
 '102909',
 '1109448',
 '1123274',
 '1132716',
 '1291274',
 '1454937',
 '1502149',
 '1535602',
 '1535631',
 '1599469',
 '1599576',
 '1610520',
 '1730578',
 '801051',
 '861177',
 '870481',
 '9015',
 '914208',
 '949509']

In [12]:
unique_urls_list=[]
for i in range(len(urls_list)):
    unique_urls=[urls_list[i][j] for j in range(len(urls_list[i])) if urls_list[i][j].split('data/')[1].split('/')[0] in unique_companies]
    unique_urls_list.append(unique_urls)

In [13]:
[print('number of filings: %s'%len(set(unique_urls_list[i]))) for i in range(len(unique_urls_list))] #some filings= 19 and others 20

number of filings: 20
number of filings: 20
number of filings: 20
number of filings: 20
number of filings: 20
number of filings: 19
number of filings: 20
number of filings: 19
number of filings: 20


[None, None, None, None, None, None, None, None, None]

In [14]:
unique_urls_list[0]

['https://www.sec.gov/Archives/edgar/data/1027796/0001567619-19-002790-index.html',
 'https://www.sec.gov/Archives/edgar/data/102909/0000950123-19-001986-index.html',
 'https://www.sec.gov/Archives/edgar/data/1109448/0001532155-19-000126-index.html',
 'https://www.sec.gov/Archives/edgar/data/1123274/0001123274-19-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1132716/0001132716-19-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1291274/0000950123-19-001436-index.html',
 'https://www.sec.gov/Archives/edgar/data/1454937/0001454937-19-000003-index.html',
 'https://www.sec.gov/Archives/edgar/data/1502149/0001567619-19-002243-index.html',
 'https://www.sec.gov/Archives/edgar/data/1535602/0001535602-19-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1535631/0001535602-19-000002-index.html',
 'https://www.sec.gov/Archives/edgar/data/1599469/0001599469-19-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1599576/0001599576-19-000001-index.

In [18]:
def parser(urls):
    '''Returns dataframe of all the securities of all filings by concatenating the filing tables found at Edgar Archives'''
    df = []
    for url in urls:
        my_header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
        #url = 'https://www.sec.gov/Archives/edgar/data/1000742/0001567619-20-003961-index.html'
        page = requests.get(url,  headers=my_header)    
        data = page.text
        soup = BeautifulSoup(data, "lxml")
        result= soup.find_all('div', attrs={'class':'formGrouping'})[0]
        date = result.find('div',attrs={'class':'info'}).text
        index = soup.find('div',{'id':'secNum'}).text.replace('SEC Accession No. ','').replace('\n','')
        entityName = soup.find('span',attrs={'class':"companyName"}).text 
        entityName = entityName.partition("(Filer)")[0]

        #get third hyperlink on page which directs to info table
        table_link = soup.find('table',attrs={'class':'tableFile'})
        table_link = table_link.find_all('a')
        table_link = table_link[2]
        identifier=table_link.get('href')
        url = 'https://www.sec.gov'+identifier
        
        #get CIK number from URL
        cik_company = url.split('data/')[1].split('/')[0]
        #parse html table to dataframe
        ## retry when 'https forbidden' error is raised, until page can be scraped successfully 
        DF_13F = None
        while DF_13F is None:
            try:
                DF_13F = pd.read_html(url)
                DF_13F = DF_13F[-1]
                DF_13F = DF_13F.iloc[2:]
                new_header = DF_13F.iloc[0]
                DF_13F.columns = new_header
                DF_13F = DF_13F.iloc[2:]
                DF_13F['date_reported'] = date
                DF_13F['CIK'] = cik_company
                DF_13F['filing index'] = index
                DF_13F['Entity name'] = entityName

                # Get entity name from CIK code from list of all funds
                #DF_13F['Entity Name'] = df_MF['Entity Name'].loc[df_MF['CIK Number'] == cik_company].values

                DF_13F = DF_13F[['filing index','date_reported','NAME OF ISSUER','TITLE OF CLASS','CUSIP','(x$1000)','PRN AMT','PRN','CIK','Entity name']]
                df.append(DF_13F)
                print('Filing found for cik %s at %s' %(cik_company,url))

            except:
                print('Retry, could not parse cik %s at: %s'%(cik_company,url))
                pass
    return pd.concat(df)

Repeat parsing in tables for each quarter

In [19]:
df_All_with2019 = {'quarter_' +str(i) : parser(unique_urls_list[i]) for i in range(4) }

Filing found for cik 1027796 at https://www.sec.gov/Archives/edgar/data/1027796/000156761919002790/xslForm13F_X01/form13fInfoTable.xml
Retry, could not parse cik 102909 at: https://www.sec.gov/Archives/edgar/data/102909/000095012319001986/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 102909 at https://www.sec.gov/Archives/edgar/data/102909/000095012319001986/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 1109448 at https://www.sec.gov/Archives/edgar/data/1109448/000153215519000126/xslForm13F_X01/ACML_13F-HR_Table.xml
Filing found for cik 1123274 at https://www.sec.gov/Archives/edgar/data/1123274/000112327419000001/xslForm13F_X01/US13F_PCO_EUR_2018_Q4.xml
Filing found for cik 1132716 at https://www.sec.gov/Archives/edgar/data/1132716/000113271619000001/xslForm13F_X01/ubsocq418infotable.xml
Filing found for cik 1291274 at https://www.sec.gov/Archives/edgar/data/1291274/000095012319001436/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 1454937 at https://www.

Filing found for cik 1132716 at https://www.sec.gov/Archives/edgar/data/1132716/000113271619000012/xslForm13F_X01/ubsocQ2_19infotable.xml
Filing found for cik 1291274 at https://www.sec.gov/Archives/edgar/data/1291274/000095012319007727/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 1454937 at https://www.sec.gov/Archives/edgar/data/1454937/000145493719000010/xslForm13F_X01/infotable.xml
Filing found for cik 1502149 at https://www.sec.gov/Archives/edgar/data/1502149/000156761919017396/xslForm13F_X01/form13fInfoTable.xml
Retry, could not parse cik 1535602 at: https://www.sec.gov/Archives/edgar/data/1535602/000159957619000007/xslForm13F_X01/US13F_PCO_2019_Q2.xml
Retry, could not parse cik 1535602 at: https://www.sec.gov/Archives/edgar/data/1535602/000159957619000007/xslForm13F_X01/US13F_PCO_2019_Q2.xml
Filing found for cik 1535602 at https://www.sec.gov/Archives/edgar/data/1535602/000159957619000007/xslForm13F_X01/US13F_PCO_2019_Q2.xml
Filing found for cik 1535631 at https://ww